<a href="https://colab.research.google.com/github/abunchoftigers/Prediction-of-Product-Sales/blob/main/Prediction_of_Product_Sales_Stack_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediction of Product Sales Part 2
[Part One](https://colab.research.google.com/github/abunchoftigers/Prediction-of-Product-Sales/blob/main/Prediction_of_Product_Sales.ipynb)

# Run the first half of this project
(Includes data cleaning)

In [ ]:
# %run '/content/drive/MyDrive/Coding Dojo - Data Science/01-Fundamentals/Colab Notebooks/Prediction_of_Product_Sales.ipynb'
# df = df

 - Author: David Dyer

## imports

In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
# import missingno

from sklearn import set_config
set_config(transform_output='pandas')

from google.colab import drive
import warnings

warnings.simplefilter('ignore')

## Define regression metrics

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
def regression_metrics(y_true, y_pred, label='', verbose = True, output_dict=False):
  # Get metrics
  mae = mean_absolute_error(y_true, y_pred)
  mse = mean_squared_error(y_true, y_pred)
  rmse = mean_squared_error(y_true, y_pred, squared=False)
  r_squared = r2_score(y_true, y_pred)
  if verbose == True:
    # Print Result with Label and Header
    header = "-"*60
    print(header, f"Regression Metrics: {label}", header, sep='\n')
    print(f"- MAE = {mae:,.3f}")
    print(f"- MSE = {mse:,.3f}")
    print(f"- RMSE = {rmse:,.3f}")
    print(f"- R^2 = {r_squared:,.3f}")
  if output_dict == True:
      metrics = {'Label':label, 'MAE':mae,
                 'MSE':mse, 'RMSE':rmse, 'R^2':r_squared}
      return metrics
def evaluate_regression(reg, X_train, y_train, X_test, y_test, verbose = True,
                        output_frame=False):
  # Get predictions for training data
  y_train_pred = reg.predict(X_train)
  # Call the helper function to obtain regression metrics for training data
  results_train = regression_metrics(y_train, y_train_pred, verbose = verbose,
                                     output_dict=output_frame,
                                     label='Training Data')
  print()
  # Get predictions for test data
  y_test_pred = reg.predict(X_test)
  # Call the helper function to obtain regression metrics for test data
  results_test = regression_metrics(y_test, y_test_pred, verbose = verbose,
                                  output_dict=output_frame,
                                    label='Test Data' )
  # Store results in a dataframe if ouput_frame is True
  if output_frame:
    results_df = pd.DataFrame([results_train,results_test])
    # Set the label as the index
    results_df = results_df.set_index('Label')
    # Set index.name to none to get a cleaner looking result
    results_df.index.name=None
    # Return the dataframe
    return results_df.round(3)

## Get the data
Uncomment only if not running Part 1 first

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
fpath = '/content/drive/MyDrive/Coding Dojo - Data Science/01-Fundamentals/Week 2/Data/sales_predictions_2023.csv'
df = pd.read_csv(fpath)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


## code

In [ ]:
#  Clean the fat column
item_fat_map = {
    'LF': 'Low Fat',
    'low fat': 'Low Fat',
    'reg': 'Regular'
}
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace(item_fat_map)
df['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

In [ ]:
# Features and target
X = df.drop(columns=['Item_Outlet_Sales', 'Item_Identifier'])
y = df['Item_Outlet_Sales']

X

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2
2,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1
3,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store
4,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1
...,...,...,...,...,...,...,...,...,...,...
8518,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1
8519,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,NaN,Tier 2,Supermarket Type1
8520,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1
8521,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2


In [ ]:
# Test train split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [ ]:
# Fill in missing string values
obj_cols = df.select_dtypes(include='object').drop(columns=['Item_Identifier']).columns
# Fill in missing numeric values
num_cols = df.select_dtypes(include='number').columns
df[num_cols] = df[num_cols].fillna(value=-1)

In [ ]:
# Now it's safe to fill in missing values
X_train[obj_cols] = X_train[obj_cols].fillna(value='MISSING')
X_train[obj_cols] = X_train[obj_cols].fillna(value='MISSING')

Numeric pipeline

In [ ]:
scaler = StandardScaler()
mean_imputer = SimpleImputer(strategy="mean")

numeric_pipe = make_pipeline(mean_imputer, scaler)
numeric_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

Categorical pipeline

In [ ]:
impute_missing = SimpleImputer(strategy='constant',fill_value='MISSING')
ohe_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

ohe_encoder.fit(X_train)

ohe_pipe = make_pipeline(impute_missing, ohe_encoder)

Numeric pipeline

Ordinal pipeline

In [ ]:
ord_cols = df[['Item_Fat_Content', 'Outlet_Establishment_Year', 'Outlet_Size']]
ord_cols

,Item_Fat_Content,Outlet_Establishment_Year,Outlet_Size
0,Low Fat,1999,Medium
1,Regular,2009,Medium
2,Low Fat,1999,Medium
3,Regular,1998,NaN
4,Low Fat,1987,High
...,...,...,...
8518,Low Fat,1987,High
8519,Regular,2002,NaN
8520,Low Fat,2004,Small
8521,Regular,2009,Medium


Create preprocessing object

In [ ]:
num_tuple = ('numeric', numeric_pipe, num_cols)
ohe_tuple = ('categorical', ohe_pipe, obj_cols)

In [ ]:
col_transformer = ColumnTransformer([num_tuple, ohe_tuple], verbose_feature_names_out=False)

In [ ]:
# col_transformer.fit(X_train)

# X_train

# Project 1 - Part 6 (Core):
This week, you will add modeling to your sales prediction project. The goal of this is to help the retailer understand the properties of products and outlets that play crucial roles in predicting sales.


**CRISP-DM Phase 4 - Modeling**

1. Your first task is to build a linear regression model to predict sales.

 * Build a linear regression model.
 * Use the custom evaluation function to get the metrics for your model (on training and test data).
 * Compare the training vs. test R-squared values and answer the question: to what extent is this model overfit/underfit?


In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
linreg_pipe = make_pipeline(col_transformer, lin_reg)

In [ ]:
# This would be a good time for preprocessing
ohe_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

ohe_encoder.fit(X_train)

ohe_pipe = make_pipeline(ohe_encoder)
ohe_tuple = ('categorical', ohe_pipe, obj_cols)

# col_transformer = ColumnTransformer([ohe_tuple], verbose_feature_names=False)

In [ ]:
linreg_pipe = make_pipeline(ohe_pipe, lin_reg)

linreg_pipe.fit(X_train, y_train)

ValueError: ignored

In [ ]:
evaluate_regression(linreg_pipe, X_train, y_train, X_test, y_test)

------------------------------------------------------------
Regression Metrics: Training Data
------------------------------------------------------------
- MAE = 0.000
- MSE = 0.000
- RMSE = 0.000
- R^2 = 1.000

------------------------------------------------------------
Regression Metrics: Test Data
------------------------------------------------------------
- MAE = 1,119.028
- MSE = 2,230,014.113
- RMSE = 1,493.323
- R^2 = 0.192



2. Your second task is to build a Random Forest model to predict sales.

 * Build a default Random Forest model.
Use the custom evaluation function to get the metrics for your model (on training and test data).
 * Compare the training vs. test R-squared values and answer the question: to what extent is this model overfit/underfit?
 * Compare this model's performance to the linear regression model: which model has the best test scores?



3. Use GridSearchCV to tune at least two hyperparameters for a Random Forest model.

 * After determining the best parameters from your GridSearch, fit and evaluate a final best model on the entire training set (no folds).
 * Compare your tuned model to your default Random Forest: did the performance improve?



**CRISP-DM Phase 5 - Evaluation**

4. You now have tried several different models on your data set. You need to determine which model to implement.

 * Overall, which model do you recommend?
 * Justify your recommendation.
 * In a Markdown cell:
    * Interpret your model's performance based on R-squared in a way that your non-technical stakeholder can understand.
    * Select another regression metric (RMSE/MAE/MSE) to express the performance of your model to your stakeholder.
   * Include why you selected this metric to explain to your stakeholder.
   * Compare the training vs. test scores and answer the question: to what extent is this model overfit/underfit?